In [110]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import os
import re

In [111]:
html = open('search_results.html')
soup = BeautifulSoup(html, 'html.parser')

In [112]:
# Extract all links inside result-item medicine divs
urls = [a['href'] for a in soup.select('.result-item.medicine a[href]')]

In [113]:
base_url ='https://www.farmacotherapeutischkompas.nl'

In [114]:
# Function to fetch and extract full 'recipe' sections containing ' FNA'
def extract_fna_recipe(url):
    full_url = base_url + url
    filename= url.split('/')[-1]
    filepath = f'pages/{filename}.html'       
    try:
        if not os.path.exists(filepath):
            try:
                response = requests.get(full_url)
                file = open(filepath, 'wb')
                file.write(response.content)
                file.close()

            except Exception as e:
                print(e)
            if response.status_code == 200:
                page_soup = BeautifulSoup(response.text, 'html.parser')
            
        else:
            try:
                file = open(filepath)
                page_soup = BeautifulSoup(file.read(), 'html.parser')
                file.close()
            except Exception as e:
                print(e)
            
        fna_recipes = [section for section in page_soup.find_all('section', class_='recipe') if 'FNA' in section.get_text()]

        atc = page_soup.find_all('span',class_='byline-item')
        atc_text = atc[-1].get_text() if atc else ""
            
        return fna_recipes, atc_text
    
    except Exception as e:
        print(e)
        return [],""

In [115]:
def get_ATC():
    pass

In [116]:
columns = ["Brandname", "ActiveIngredient", "Strength", "DosageForm", "ATC", "PrescribingProduct"]

#Initialize library
records = []

# Iterate through URLs and extract relevant content

for url in urls:
    print(base_url+url)
    fna_recipes, atc = extract_fna_recipe(url)

    print(atc)
    #print(fna_recipes)
    for section in fna_recipes:
        name = section.find("span", class_="name").text.strip() if section.find("span", class_="name") else ""
        print(name)

        try:
            doses = [dose for dose in section.find_all(class_='doses')]
            #print(doses)

            for dose in doses:
                try:
                    dosage_form = dose.find("dd", title="Toedieningsvorm").text.strip() if dose.find("dd", title="Toedieningsvorm") else ""
                    print(dosage_form)
                except Exception as e:
                    print(e)
                            
                try:
                    strengths =  dose.find("dd", title="Sterkte").text.strip().split(',')
                    #strenghts = re.findall(r'\d(?:,\d+)?(?:\s*[a-zA-Z]+)?', strengths_text)

                    print(strengths)
                except Exception as e:
                    print(e)
                    strengths=[]
                
                if strengths:
                    for strength in strengths:
                        print(strength)
                        records.append({
                            "Brandname": "",
                            "ActiveIngredient": name,
                            "Strength": strength,
                            "DosageForm": dosage_form,
                            "ATC": atc,
                            "PrescribingProduct": name + " - " + strength + " - " + dosage_form
                        })
                else:
                    records.append({
                        "Brandname": "",
                        "ActiveIngredient": name,
                        "Strength": "",
                        "DosageForm": dosage_form,
                        "ATC": atc,
                        "PrescribingProduct": name + " - " + "" + " - " + dosage_form
                    })
        ####################################################
        except Exception as e:
            print(e)
        
            try:
                dosage_form = section.find("dd", title="Toedieningsvorm").text.strip() if section.find("dd", title="Toedieningsvorm") else ""
                #dosage_form = section.find_all("dd", title="Toedieningsvorm")

                print(dosage_form)
            except Exception as e:
                print(e)
                        
            try:
                strengths=  dose.find("dd", title="Sterkte").text.strip().split(',')
                #strenghts = re.findall(r'\d+(?:,\d+)?(?:\s*[a-zA-Z]+)?', strengths_text)
                print(strengths)
            except Exception as e:
                print(e)
                strengths=[]
            
            if strengths:
                for strength in strengths:
                    print(strength)
                    records.append({
                        "Brandname": "",
                        "ActiveIngredient": name,
                        "Strength": strength,
                        "DosageForm": dosage_form,
                        "ATC": atc,
                        "PrescribingProduct": name + " - " + strength + " - " + dosage_form
                    })
            else:
                records.append({
                    "Brandname": "",
                    "ActiveIngredient": name,
                    "Strength": "",
                    "DosageForm": dosage_form,
                    "ATC": atc,
                    "PrescribingProduct": name + " - " + "" + " - " + dosage_form
                })

#Create dataframe
df = pd.DataFrame(records, columns=columns)
# Display DataFrame
display(df)





https://www.farmacotherapeutischkompas.nl/bladeren/preparaatteksten/h/hydrocortison__systemisch_
H02AB09
Hydrocortison FNA
Capsule
['1 mg', ' 2 mg', ' 3 mg', ' 4 mg', ' 5 mg', ' 6 mg', ' 7 mg', ' 8 mg', ' 9 mg', ' 10 mg']
1 mg
 2 mg
 3 mg
 4 mg
 5 mg
 6 mg
 7 mg
 8 mg
 9 mg
 10 mg
https://www.farmacotherapeutischkompas.nl/bladeren/groepsteksten/corticosteroiden__systemisch

https://www.farmacotherapeutischkompas.nl/bladeren/preparaatteksten/c/colecalciferol
A11CC05
Colecalciferol drank FNA
Drank
['50.000 IE/ml']
50.000 IE/ml
https://www.farmacotherapeutischkompas.nl/bladeren/groepsteksten/vitamine_d_en_analoga

https://www.farmacotherapeutischkompas.nl/bladeren/preparaatteksten/i/indifferente__vette__creme
D02A
Cetomacrogolcrème FNA
Crème
'NoneType' object has no attribute 'text'
Cetomacrogolcrème met vaseline FNA
Vaselinecrème
['10 %', ' 20 %', ' 50 %']
10 %
 20 %
 50 %
Lanettecrème I/II FNA
Crème
'NoneType' object has no attribute 'text'
Lanettecrème met vaseline FNA
Vaselinecrème
['

,Brandname,ActiveIngredient,Strength,DosageForm,ATC,PrescribingProduct
0,,Hydrocortison FNA,1 mg,Capsule,H02AB09,Hydrocortison FNA - 1 mg - Capsule
1,,Hydrocortison FNA,2 mg,Capsule,H02AB09,Hydrocortison FNA - 2 mg - Capsule
2,,Hydrocortison FNA,3 mg,Capsule,H02AB09,Hydrocortison FNA - 3 mg - Capsule
3,,Hydrocortison FNA,4 mg,Capsule,H02AB09,Hydrocortison FNA - 4 mg - Capsule
4,,Hydrocortison FNA,5 mg,Capsule,H02AB09,Hydrocortison FNA - 5 mg - Capsule
...,...,...,...,...,...,...
356,,Triamcinolon-Salicylzuur Crème/Oplossing/Smeer...,,"Oplossing (0,1%/2%)",D07XB02,Triamcinolon-Salicylzuur Crème/Oplossing/Smeer...
357,,Triamcinolon-Salicylzuur Crème/Oplossing/Smeer...,,"Smeersel (0,1%/5%)",D07XB02,Triamcinolon-Salicylzuur Crème/Oplossing/Smeer...
358,,Zure oordruppels met Hydrocortison 1% FNA,,Oordruppels,S02CA03,Zure oordruppels met Hydrocortison 1% FNA - -...
359,,Dexamethason/Chlooramfenicol/Polymyxine B oord...,,Oordruppels,S02CA06,Dexamethason/Chlooramfenicol/Polymyxine B oord...


In [117]:
from datetime import date
output_filename = f'edups_FNA_formularium_v{date.today()}.csv'

df.to_csv(output_filename, index=False, encoding='latin-1',sep=';', decimal=',', errors='ignore')
